In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Usable Code

In [2]:
!pwd

/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor


In [3]:
files = sorted([f for f in os.listdir('csv') if '.csv' in f])

In [4]:
files

['07-08.csv',
 '08-09.csv',
 '09-10.csv',
 '10-11.csv',
 '11-12.csv',
 '12-13.csv',
 '13-14.csv',
 '14-15.csv',
 '15-16.csv',
 '16-17.csv',
 '17-18.csv',
 '18-19.csv']

In [170]:
def clean_odds(file):
    '''
    cleans csv file and returns dataframe object that contains the over/under lines (target)
    '''
    df = pd.read_csv(file)
    df = df.iloc[:,:13] #some files contained unnamed extra rows
    df.dropna(axis=0,inplace=True) #and others columns
    
    #makes a list of game_id's equivalent to how many games were played that season
    mylist = []
    for i in range(1, int(df.shape[0]/2 + 1)):
        mylist.append(i)
        mylist.append(i)
    df['game_id'] = mylist #maps game_id's to games (spread across 2 rows)
    
    df = df.merge(df, on='game_id', suffixes=(None,'_2')) #gets games to be in one row (creates 4/game)
    df = df[1::4] #gets correct mapping
    df.reset_index(drop=True, inplace=True)
    
    #make dates include year so we can join with another dataset later
    df['Date'] = df['Date'].astype('int')
    df['Date'] = df['Date'].astype('str')
    #df['Date'] = df['Date'].str.strip('.0')
    dates = []
    for date in df['Date']:
        if int(date) > 1000:
            date = '20'+file[:2]+date #GET FILENAMES AND USE HERE
            dates.append(date)
        else:
            date = '20'+file[3:5]+'0'+date #GET FILENAMES AND USE HERE
            dates.append(date)
    df['Date'] = dates
    
    #take out pick-em's and replace with zero for later transformation
    df['Open'].replace(['PK','pk'],'0',inplace=True)
    df['Close'].replace(['PK','pk'],'0',inplace=True)
    df['Open_2'].replace(['PK','pk'],'0',inplace=True)
    df['Close_2'].replace(['PK','pk'],'0',inplace=True)
    df.replace('197.5u10','197.5',inplace=True) #one unique occurrence
    #map strings as floats for comparison
    df['Open'] = df['Open'].astype('float')
    df['Open_2'] = df['Open_2'].astype('float')
    df['Close'] = df['Close'].astype('float')
    df['Close_2'] = df['Close_2'].astype('float')
    #get correct over/under line at opening and closing of sportsbook
    df['O/U_open'] = np.where(df['Open'] > df['Open_2'],df['Open'],df['Open_2'])
    df['O/U_close'] = np.where(df['Close'] > df['Close_2'],df['Close'],df['Close_2'])
    
    #rename columns to correct home/visitor 
    df['Visitor'] = df['Team']
    df['Home'] = df['Team_2']
    
    #add season column
    df['Season'] = file[:2] + file[3:5]
    
    #drop unneeded info
    df = df[['Date','Home','Visitor','O/U_open','O/U_close','Season']]
    return df

Use this clean_odds function to generate our dataset to combine with our scraped game data.

In [165]:
for file in files:
    print(file[:2]+file[3:5])

0708
0809
0910
1011
1112
1213
1314
1415
1516
1617
1718
1819


In [171]:
cd csv

[Errno 2] No such file or directory: 'csv'
/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor/csv


In [172]:
totals = pd.concat([clean_odds(f) for f in files])

In [173]:
totals

,Date,Home,Visitor,O/U_open,O/U_close,Season
0,20071030,SanAntonio,Portland,184.0,189.5,0708
1,20071030,GoldenState,Utah,214.5,212.0,0708
2,20071030,LALakers,Houston,191.0,199.0,0708
3,20071031,Toronto,Philadelphia,190.0,191.0,0708
4,20071031,Indiana,Washington,200.0,203.5,0708
...,...,...,...,...,...,...
1307,20190602,Toronto,GoldenState,216.0,213.5,1819
1308,20190605,GoldenState,Toronto,214.0,209.5,1819
1309,20190607,GoldenState,Toronto,216.0,215.0,1819
1310,20190610,Toronto,GoldenState,212.0,217.0,1819


In [174]:
totals.reset_index(drop=True,inplace=True)
totals

,Date,Home,Visitor,O/U_open,O/U_close,Season
0,20071030,SanAntonio,Portland,184.0,189.5,0708
1,20071030,GoldenState,Utah,214.5,212.0,0708
2,20071030,LALakers,Houston,191.0,199.0,0708
3,20071031,Toronto,Philadelphia,190.0,191.0,0708
4,20071031,Indiana,Washington,200.0,203.5,0708
...,...,...,...,...,...,...
15515,20190602,Toronto,GoldenState,216.0,213.5,1819
15516,20190605,GoldenState,Toronto,214.0,209.5,1819
15517,20190607,GoldenState,Toronto,216.0,215.0,1819
15518,20190610,Toronto,GoldenState,212.0,217.0,1819


Let's pickle this initial processing of the csv's and subsequent DataFrame. We can always comeback to this if we need it.

In [175]:
with open('lines.pickle', 'wb') as to_write:
    pickle.dump(totals, to_write)

In [14]:
pwd

'/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor/csv'

In [15]:
cd ..

/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor


In [16]:
with open('game_df.pickle','rb') as read_file:
    game_df = pickle.load(read_file)
    
game_df

,game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,...,ts_per_h,threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h
0,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,...,.538,.276,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7
1,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,...,.512,.299,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3
2,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,...,.485,.105,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1
3,200710310IND,229,WAS,IND,98.4,98.4,.394,.484,13.2,13.6,...,.552,.315,.391,64.6,50.0,52.6,11.0,7.6,100.0,101.2
4,200710310ORL,185,MIL,ORL,88.1,88.1,.402,.521,12.6,10.2,...,.577,.301,.479,64.7,46.7,46.9,2.3,12.9,100.0,94.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15516,201906020TOR,213,GSW,TOR,100.0,100.0,.543,.431,14.0,12.5,...,.493,.404,.277,85.0,53.8,48.6,8.0,4.2,100.0,109.0
15517,201906050GSW,232,TOR,GSW,99.6,99.6,.628,.462,13.3,11.8,...,.523,.396,.330,84.8,50.6,69.4,8.0,6.8,100.0,123.5
15518,201906070GSW,197,TOR,GSW,94.8,94.8,.477,.500,8.5,16.3,...,.527,.346,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7
15519,201906100TOR,211,GSW,TOR,94.3,94.3,.585,.494,14.5,11.8,...,.542,.376,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4


In [19]:
team_abbrev = game_df['home'].unique()
team_abbrev

array(['SAS', 'GSW', 'LAL', 'IND', 'ORL', 'TOR', 'NJN', 'CLE', 'MEM',
       'NOH', 'DEN', 'MIA', 'UTA', 'SEA', 'CHA', 'ATL', 'BOS', 'MIN',
       'CHI', 'LAC', 'PHO', 'PHI', 'WAS', 'DAL', 'HOU', 'MIL', 'NYK',
       'DET', 'SAC', 'POR', 'OKC', 'BRK', 'NOP', 'CHO'], dtype=object)

In [20]:
len(team_abbrev)

34

In [21]:
totals['Home'].unique()

array(['SanAntonio', 'GoldenState', 'LALakers', 'Toronto', 'Indiana',
       'Orlando', 'NewJersey', 'Cleveland', 'Memphis', 'NewOrleans',
       'Denver', 'Miami', 'Utah', 'Seattle', 'Charlotte', 'Atlanta',
       'Boston', 'Minnesota', 'Chicago', 'Phoenix', 'LAClippers',
       'Philadelphia', 'Washington', 'Milwaukee', 'Houston', 'Dallas',
       'NewYork', 'Detroit', 'Sacramento', 'Portland', 'OklahomaCity',
       'Brooklyn'], dtype=object)

In [22]:
team_dict = {
    'SAS': 'SanAntonio',
    'GSW': 'GoldenState',
    'LAL': 'LALakers',
    'TOR': 'Toronto',
    'IND': 'Indiana',
    'ORL': 'Orlando',
    'NJN': 'NewJersey',
    'CLE': 'Cleveland',
    'MEM': 'Memphis',
    'NOH': 'NewOrleans',
    'NOP': 'NewOrleans',
    'DEN': 'Denver',
    'MIA': 'Miami',
    'UTA': 'Utah',
    'SEA': 'Seattle',
    'CHA': 'Charlotte',
    'CHO': 'Charlotte',
    'ATL': 'Atlanta',
    'BOS': 'Boston',
    'MIN': 'Minnesota',
    'CHI': 'Chicago',
    'PHO': 'Phoenix',
    'LAC': 'LAClippers',
    'PHI': 'Philadelphia',
    'WAS': 'Washington',
    'MIL': 'Milwaukee',
    'HOU': 'Houston',
    'DAL': 'Dallas',
    'NYK': 'NewYork',
    'DET': 'Detroit',
    'SAC': 'Sacramento',
    'POR': 'Portland',
    'OKC': 'OklahomaCity',
    'BRK': 'Brooklyn',
}

In [23]:
game_df['home'].map(team_dict)

0         SanAntonio
1        GoldenState
2           LALakers
3            Indiana
4            Orlando
            ...     
15516        Toronto
15517    GoldenState
15518    GoldenState
15519        Toronto
15520    GoldenState
Name: home, Length: 15521, dtype: object

In [24]:
game_df['home_team'] = game_df['home'].map(team_dict)

In [25]:
game_df

,game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,...,threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,home_team
0,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,...,.276,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio
1,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,...,.299,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState
2,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,...,.105,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1,LALakers
3,200710310IND,229,WAS,IND,98.4,98.4,.394,.484,13.2,13.6,...,.315,.391,64.6,50.0,52.6,11.0,7.6,100.0,101.2,Indiana
4,200710310ORL,185,MIL,ORL,88.1,88.1,.402,.521,12.6,10.2,...,.301,.479,64.7,46.7,46.9,2.3,12.9,100.0,94.2,Orlando
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15516,201906020TOR,213,GSW,TOR,100.0,100.0,.543,.431,14.0,12.5,...,.404,.277,85.0,53.8,48.6,8.0,4.2,100.0,109.0,Toronto
15517,201906050GSW,232,TOR,GSW,99.6,99.6,.628,.462,13.3,11.8,...,.396,.330,84.8,50.6,69.4,8.0,6.8,100.0,123.5,GoldenState
15518,201906070GSW,197,TOR,GSW,94.8,94.8,.477,.500,8.5,16.3,...,.346,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7,GoldenState
15519,201906100TOR,211,GSW,TOR,94.3,94.3,.585,.494,14.5,11.8,...,.376,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4,Toronto


In [26]:
game_df['game_date'] = game_df['game_id'].apply(lambda x: x[:8])

In [27]:
game_df

,game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,...,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,home_team,game_date
0,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,...,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio,20071030
1,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,...,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState,20071030
2,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,...,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1,LALakers,20071030
3,200710310IND,229,WAS,IND,98.4,98.4,.394,.484,13.2,13.6,...,.391,64.6,50.0,52.6,11.0,7.6,100.0,101.2,Indiana,20071031
4,200710310ORL,185,MIL,ORL,88.1,88.1,.402,.521,12.6,10.2,...,.479,64.7,46.7,46.9,2.3,12.9,100.0,94.2,Orlando,20071031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15516,201906020TOR,213,GSW,TOR,100.0,100.0,.543,.431,14.0,12.5,...,.277,85.0,53.8,48.6,8.0,4.2,100.0,109.0,Toronto,20190602
15517,201906050GSW,232,TOR,GSW,99.6,99.6,.628,.462,13.3,11.8,...,.330,84.8,50.6,69.4,8.0,6.8,100.0,123.5,GoldenState,20190605
15518,201906070GSW,197,TOR,GSW,94.8,94.8,.477,.500,8.5,16.3,...,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7,GoldenState,20190607
15519,201906100TOR,211,GSW,TOR,94.3,94.3,.585,.494,14.5,11.8,...,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4,Toronto,20190610


In [28]:
game_df['id'] = game_df['game_date'] + game_df['home_team']

In [29]:
game_df

,game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,...,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,home_team,game_date,id
0,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,...,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio,20071030,20071030SanAntonio
1,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,...,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState,20071030,20071030GoldenState
2,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,...,68.4,43.0,56.3,17.2,5.8,100.0,102.1,LALakers,20071030,20071030LALakers
3,200710310IND,229,WAS,IND,98.4,98.4,.394,.484,13.2,13.6,...,64.6,50.0,52.6,11.0,7.6,100.0,101.2,Indiana,20071031,20071031Indiana
4,200710310ORL,185,MIL,ORL,88.1,88.1,.402,.521,12.6,10.2,...,64.7,46.7,46.9,2.3,12.9,100.0,94.2,Orlando,20071031,20071031Orlando
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15516,201906020TOR,213,GSW,TOR,100.0,100.0,.543,.431,14.0,12.5,...,85.0,53.8,48.6,8.0,4.2,100.0,109.0,Toronto,20190602,20190602Toronto
15517,201906050GSW,232,TOR,GSW,99.6,99.6,.628,.462,13.3,11.8,...,84.8,50.6,69.4,8.0,6.8,100.0,123.5,GoldenState,20190605,20190605GoldenState
15518,201906070GSW,197,TOR,GSW,94.8,94.8,.477,.500,8.5,16.3,...,82.9,51.9,74.3,6.3,11.1,100.0,110.7,GoldenState,20190607,20190607GoldenState
15519,201906100TOR,211,GSW,TOR,94.3,94.3,.585,.494,14.5,11.8,...,83.3,53.8,50.0,6.4,12.5,100.0,112.4,Toronto,20190610,20190610Toronto


In [176]:
totals['id'] = totals['Date'] + totals['Home']

In [177]:
totals

,Date,Home,Visitor,O/U_open,O/U_close,Season,id
0,20071030,SanAntonio,Portland,184.0,189.5,0708,20071030SanAntonio
1,20071030,GoldenState,Utah,214.5,212.0,0708,20071030GoldenState
2,20071030,LALakers,Houston,191.0,199.0,0708,20071030LALakers
3,20071031,Toronto,Philadelphia,190.0,191.0,0708,20071031Toronto
4,20071031,Indiana,Washington,200.0,203.5,0708,20071031Indiana
...,...,...,...,...,...,...,...
15515,20190602,Toronto,GoldenState,216.0,213.5,1819,20190602Toronto
15516,20190605,GoldenState,Toronto,214.0,209.5,1819,20190605GoldenState
15517,20190607,GoldenState,Toronto,216.0,215.0,1819,20190607GoldenState
15518,20190610,Toronto,GoldenState,212.0,217.0,1819,20190610Toronto


In [178]:
totals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15520 entries, 0 to 15519
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       15520 non-null  object 
 1   Home       15520 non-null  object 
 2   Visitor    15520 non-null  object 
 3   O/U_open   15520 non-null  float64
 4   O/U_close  15520 non-null  float64
 5   Season     15520 non-null  object 
 6   id         15520 non-null  object 
dtypes: float64(2), object(5)
memory usage: 848.9+ KB


In [33]:
game_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15521 entries, 0 to 15520
Data columns (total 71 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   game_id       15521 non-null  object
 1   total         15521 non-null  int64 
 2   away          15521 non-null  object
 3   home          15521 non-null  object
 4   pace_v        15521 non-null  object
 5   pace_h        15521 non-null  object
 6   eFg_v         15521 non-null  object
 7   eFg_h         15521 non-null  object
 8   tov_v         15521 non-null  object
 9   tov_h         15521 non-null  object
 10  orb_v         15521 non-null  object
 11  orb_h         15521 non-null  object
 12  ft_fga_v      15521 non-null  object
 13  ft_fga_h      15521 non-null  object
 14  ortg_v        15521 non-null  object
 15  ortg_h        15521 non-null  object
 16  fg_v          15521 non-null  object
 17  fga_v         15521 non-null  object
 18  fg_per_v      15521 non-null  object
 19  thre

Now both databases have a column to merge on. The 'id' column.

In [179]:
df = pd.merge(totals,game_df,on='id')

In [180]:
df

,Date,Home,Visitor,O/U_open,O/U_close,Season,id,game_id,total,away,...,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,home_team,game_date
0,20071030,SanAntonio,Portland,184.0,189.5,0708,20071030SanAntonio,200710300SAS,203,POR,...,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio,20071030
1,20071030,GoldenState,Utah,214.5,212.0,0708,20071030GoldenState,200710300GSW,213,UTA,...,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState,20071030
2,20071030,LALakers,Houston,191.0,199.0,0708,20071030LALakers,200710300LAL,188,HOU,...,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1,LALakers,20071030
3,20071031,Toronto,Philadelphia,190.0,191.0,0708,20071031Toronto,200710310TOR,203,PHI,...,.200,64.3,44.6,56.1,8.8,6.2,100.0,106.8,Toronto,20071031
4,20071031,Indiana,Washington,200.0,203.5,0708,20071031Indiana,200710310IND,229,WAS,...,.391,64.6,50.0,52.6,11.0,7.6,100.0,101.2,Indiana,20071031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15010,20190602,Toronto,GoldenState,216.0,213.5,1819,20190602Toronto,201906020TOR,213,GSW,...,.277,85.0,53.8,48.6,8.0,4.2,100.0,109.0,Toronto,20190602
15011,20190605,GoldenState,Toronto,214.0,209.5,1819,20190605GoldenState,201906050GSW,232,TOR,...,.330,84.8,50.6,69.4,8.0,6.8,100.0,123.5,GoldenState,20190605
15012,20190607,GoldenState,Toronto,216.0,215.0,1819,20190607GoldenState,201906070GSW,197,TOR,...,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7,GoldenState,20190607
15013,20190610,Toronto,GoldenState,212.0,217.0,1819,20190610Toronto,201906100TOR,211,GSW,...,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4,Toronto,20190610


In [181]:
df.shape

(15015, 77)

In [182]:
df.drop(columns=['Home','Visitor','game_id','home_team','game_date'],inplace=True)

In [183]:
df

,Date,O/U_open,O/U_close,Season,id,total,away,home,pace_v,pace_h,...,ts_per_h,threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h
0,20071030,184.0,189.5,0708,20071030SanAntonio,203,POR,SAS,91.8,91.8,...,.538,.276,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7
1,20071030,214.5,212.0,0708,20071030GoldenState,213,UTA,GSW,105.1,105.1,...,.512,.299,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3
2,20071030,191.0,199.0,0708,20071030LALakers,188,HOU,LAL,93.0,93.0,...,.485,.105,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1
3,20071031,190.0,191.0,0708,20071031Toronto,203,PHI,TOR,90.9,90.9,...,.573,.188,.200,64.3,44.6,56.1,8.8,6.2,100.0,106.8
4,20071031,200.0,203.5,0708,20071031Indiana,229,WAS,IND,98.4,98.4,...,.552,.315,.391,64.6,50.0,52.6,11.0,7.6,100.0,101.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15010,20190602,216.0,213.5,1819,20190602Toronto,213,GSW,TOR,100.0,100.0,...,.493,.404,.277,85.0,53.8,48.6,8.0,4.2,100.0,109.0
15011,20190605,214.0,209.5,1819,20190605GoldenState,232,TOR,GSW,99.6,99.6,...,.523,.396,.330,84.8,50.6,69.4,8.0,6.8,100.0,123.5
15012,20190607,216.0,215.0,1819,20190607GoldenState,197,TOR,GSW,94.8,94.8,...,.527,.346,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7
15013,20190610,212.0,217.0,1819,20190610Toronto,211,GSW,TOR,94.3,94.3,...,.542,.376,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4


Now that our data is fully processed, we are ready to process this in a classification algorithm

In [ ]:
with open('df.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)

## Scratch Work

In [ ]:
df = pd.read_csv(files[2])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.iloc[:,:13]

In [ ]:
df.info()

In [ ]:
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)

In [ ]:
df.shape[0]

In [ ]:
mylist = []
for i in range(1, int(df.shape[0]/2 + 1)):
    mylist.append(i)
    mylist.append(i)
print(mylist[0],mylist[-1])

In [ ]:
len(range(1,int(df.shape[0]/2 + 1)))

In [ ]:
df['game_id'] = mylist

In [ ]:
df.dropna(axis=0,inplace=True)

In [ ]:
df.info()

In [ ]:
df = df.merge(df, on='game_id', suffixes=(None,'_2'))

In [ ]:
df

In [ ]:
df = df[1::4]
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [ ]:
#sanity check that every game only appears once
df['game_id']

In [ ]:
df['Date'] = df['Date'].astype('int')
df['Date']

In [ ]:
df['Date'] = df['Date'].astype('str')
df['Date']

In [ ]:
df['Date'] = df['Date'].str.strip('.0')

In [ ]:
df['Date'].head

In [ ]:
dates = []
for date in df['Date']:
    if int(date) > 1000:
        date = '20'+files[2][:2]+date
        dates.append(date)
    else:
        date = '20'+files[2][3:5]+'0'+date
        dates.append(date)
df['Date'] = dates

In [ ]:
df['Date']

In [ ]:
df

In [ ]:
df['Visitor'] = df['Team']
df['Home'] = df['Team_2']
df

In [ ]:
df['Open'].replace('pk','0',inplace=True)
df['Close'].replace('pk','0',inplace=True)
df['Open_2'].replace('pk','0',inplace=True)
df['Close_2'].replace('pk','0',inplace=True)
df.replace('197.5u10','197.5',inplace=True)

In [ ]:
mask = df['Open'].values == 'pk'
df[mask]

In [ ]:
df['Open'] = df['Open'].astype('float')
df['Open_2'] = df['Open_2'].astype('float')
df['Close'] = df['Close'].astype('float')
df['Close_2'] = df['Close_2'].astype('float')

In [ ]:
df['O/U_open'] = np.where(df['Open'] > df['Open_2'],df['Open'],df['Open_2'])
df

In [ ]:
df['O/U_close'] = np.where(df['Close'] > df['Close_2'],df['Close'],df['Close_2'])
df

Sanity check that opening and closing lines operations worked successfully.
So long as numbers are around 200, we know we are ok.

In [ ]:
df['O/U_open'].min()

In [ ]:
df['O/U_close'].min()

In [ ]:
df = df[['Date','Home','Visitor','O/U_open','O/U_close']]
df

### Processing our lines df

First, and most importantly, we must create our target labels. This will be done by comparing the total the the opening and closing lines, and mapping that result to one of three categories: Over, Under, or Push. This will represent the winning result of that game.

In [184]:
df

,Date,O/U_open,O/U_close,Season,id,total,away,home,pace_v,pace_h,...,ts_per_h,threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h
0,20071030,184.0,189.5,0708,20071030SanAntonio,203,POR,SAS,91.8,91.8,...,.538,.276,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7
1,20071030,214.5,212.0,0708,20071030GoldenState,213,UTA,GSW,105.1,105.1,...,.512,.299,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3
2,20071030,191.0,199.0,0708,20071030LALakers,188,HOU,LAL,93.0,93.0,...,.485,.105,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1
3,20071031,190.0,191.0,0708,20071031Toronto,203,PHI,TOR,90.9,90.9,...,.573,.188,.200,64.3,44.6,56.1,8.8,6.2,100.0,106.8
4,20071031,200.0,203.5,0708,20071031Indiana,229,WAS,IND,98.4,98.4,...,.552,.315,.391,64.6,50.0,52.6,11.0,7.6,100.0,101.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15010,20190602,216.0,213.5,1819,20190602Toronto,213,GSW,TOR,100.0,100.0,...,.493,.404,.277,85.0,53.8,48.6,8.0,4.2,100.0,109.0
15011,20190605,214.0,209.5,1819,20190605GoldenState,232,TOR,GSW,99.6,99.6,...,.523,.396,.330,84.8,50.6,69.4,8.0,6.8,100.0,123.5
15012,20190607,216.0,215.0,1819,20190607GoldenState,197,TOR,GSW,94.8,94.8,...,.527,.346,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7
15013,20190610,212.0,217.0,1819,20190610Toronto,211,GSW,TOR,94.3,94.3,...,.542,.376,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4


In [185]:
df['ou1'] = df['total'] - df['O/U_open']
df['ou2'] = df['total'] - df['O/U_close']

In [186]:
df

,Date,O/U_open,O/U_close,Season,id,total,away,home,pace_v,pace_h,...,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,ou1,ou2
0,20071030,184.0,189.5,0708,20071030SanAntonio,203,POR,SAS,91.8,91.8,...,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,19.0,13.5
1,20071030,214.5,212.0,0708,20071030GoldenState,213,UTA,GSW,105.1,105.1,...,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,-1.5,1.0
2,20071030,191.0,199.0,0708,20071030LALakers,188,HOU,LAL,93.0,93.0,...,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1,-3.0,-11.0
3,20071031,190.0,191.0,0708,20071031Toronto,203,PHI,TOR,90.9,90.9,...,.200,64.3,44.6,56.1,8.8,6.2,100.0,106.8,13.0,12.0
4,20071031,200.0,203.5,0708,20071031Indiana,229,WAS,IND,98.4,98.4,...,.391,64.6,50.0,52.6,11.0,7.6,100.0,101.2,29.0,25.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15010,20190602,216.0,213.5,1819,20190602Toronto,213,GSW,TOR,100.0,100.0,...,.277,85.0,53.8,48.6,8.0,4.2,100.0,109.0,-3.0,-0.5
15011,20190605,214.0,209.5,1819,20190605GoldenState,232,TOR,GSW,99.6,99.6,...,.330,84.8,50.6,69.4,8.0,6.8,100.0,123.5,18.0,22.5
15012,20190607,216.0,215.0,1819,20190607GoldenState,197,TOR,GSW,94.8,94.8,...,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7,-19.0,-18.0
15013,20190610,212.0,217.0,1819,20190610Toronto,211,GSW,TOR,94.3,94.3,...,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4,-1.0,-6.0


In [187]:
def over_under(ou):
    if ou > 0:
        return 1
    elif ou == 0:
        return 2
    else:
        return 0

In [188]:
df['Over/Under_open'] = df['ou1'].apply(over_under)
df['Over/Under_close'] = df['ou2'].apply(over_under)

In [189]:
df

,Date,O/U_open,O/U_close,Season,id,total,away,home,pace_v,pace_h,...,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,ou1,ou2,Over/Under_open,Over/Under_close
0,20071030,184.0,189.5,0708,20071030SanAntonio,203,POR,SAS,91.8,91.8,...,50.0,51.2,8.7,6.2,100.0,105.7,19.0,13.5,1,1
1,20071030,214.5,212.0,0708,20071030GoldenState,213,UTA,GSW,105.1,105.1,...,39.8,59.4,7.6,11.4,100.0,111.3,-1.5,1.0,0,1
2,20071030,191.0,199.0,0708,20071030LALakers,188,HOU,LAL,93.0,93.0,...,43.0,56.3,17.2,5.8,100.0,102.1,-3.0,-11.0,0,0
3,20071031,190.0,191.0,0708,20071031Toronto,203,PHI,TOR,90.9,90.9,...,44.6,56.1,8.8,6.2,100.0,106.8,13.0,12.0,1,1
4,20071031,200.0,203.5,0708,20071031Indiana,229,WAS,IND,98.4,98.4,...,50.0,52.6,11.0,7.6,100.0,101.2,29.0,25.5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15010,20190602,216.0,213.5,1819,20190602Toronto,213,GSW,TOR,100.0,100.0,...,53.8,48.6,8.0,4.2,100.0,109.0,-3.0,-0.5,0,0
15011,20190605,214.0,209.5,1819,20190605GoldenState,232,TOR,GSW,99.6,99.6,...,50.6,69.4,8.0,6.8,100.0,123.5,18.0,22.5,1,1
15012,20190607,216.0,215.0,1819,20190607GoldenState,197,TOR,GSW,94.8,94.8,...,51.9,74.3,6.3,11.1,100.0,110.7,-19.0,-18.0,0,0
15013,20190610,212.0,217.0,1819,20190610Toronto,211,GSW,TOR,94.3,94.3,...,53.8,50.0,6.4,12.5,100.0,112.4,-1.0,-6.0,0,0


Making the decision to drop pushes as no money exchanges hands in this scenerio.

In [190]:
df = df[df['Over/Under_open'] != 2]
df = df[df['Over/Under_close'] != 2]

In [191]:
df

,Date,O/U_open,O/U_close,Season,id,total,away,home,pace_v,pace_h,...,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,ou1,ou2,Over/Under_open,Over/Under_close
0,20071030,184.0,189.5,0708,20071030SanAntonio,203,POR,SAS,91.8,91.8,...,50.0,51.2,8.7,6.2,100.0,105.7,19.0,13.5,1,1
1,20071030,214.5,212.0,0708,20071030GoldenState,213,UTA,GSW,105.1,105.1,...,39.8,59.4,7.6,11.4,100.0,111.3,-1.5,1.0,0,1
2,20071030,191.0,199.0,0708,20071030LALakers,188,HOU,LAL,93.0,93.0,...,43.0,56.3,17.2,5.8,100.0,102.1,-3.0,-11.0,0,0
3,20071031,190.0,191.0,0708,20071031Toronto,203,PHI,TOR,90.9,90.9,...,44.6,56.1,8.8,6.2,100.0,106.8,13.0,12.0,1,1
4,20071031,200.0,203.5,0708,20071031Indiana,229,WAS,IND,98.4,98.4,...,50.0,52.6,11.0,7.6,100.0,101.2,29.0,25.5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15010,20190602,216.0,213.5,1819,20190602Toronto,213,GSW,TOR,100.0,100.0,...,53.8,48.6,8.0,4.2,100.0,109.0,-3.0,-0.5,0,0
15011,20190605,214.0,209.5,1819,20190605GoldenState,232,TOR,GSW,99.6,99.6,...,50.6,69.4,8.0,6.8,100.0,123.5,18.0,22.5,1,1
15012,20190607,216.0,215.0,1819,20190607GoldenState,197,TOR,GSW,94.8,94.8,...,51.9,74.3,6.3,11.1,100.0,110.7,-19.0,-18.0,0,0
15013,20190610,212.0,217.0,1819,20190610Toronto,211,GSW,TOR,94.3,94.3,...,53.8,50.0,6.4,12.5,100.0,112.4,-1.0,-6.0,0,0


About 400 instances of a push, or about 2.7% of our original dataset. This number can be remembered for sampling or simulation purposes later.

### Stat Building

We have the same set of stats for both teams in any one game, so we can build offense/defense for both teams.

In [ ]:
'''map stats accordingly:
1. Get 2 "sets" of stats per game:

visitor: offense - _v stat avgs heading into the game; defense - _h stat avgs heading into the game
home: offese - _h stat avgs heading into the game; defense - _v stat avgs heading into the game

2. Map visitor/home stats to respective teams

3. Build dictionary of team's seasons to be further processed.

{1415: {GSW: {..game_35:{offense/defense stats},game_36:{..}

4. Process dictionary to have more stats/potential model features:

{GSW: {..game_35:{offense/defense stats averaged through 34 games},game_36:{..}

5. ?

'''

In [192]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14622 entries, 0 to 15014
Data columns (total 76 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              14622 non-null  object 
 1   O/U_open          14622 non-null  float64
 2   O/U_close         14622 non-null  float64
 3   Season            14622 non-null  object 
 4   id                14622 non-null  object 
 5   total             14622 non-null  int64  
 6   away              14622 non-null  object 
 7   home              14622 non-null  object 
 8   pace_v            14622 non-null  object 
 9   pace_h            14622 non-null  object 
 10  eFg_v             14622 non-null  object 
 11  eFg_h             14622 non-null  object 
 12  tov_v             14622 non-null  object 
 13  tov_h             14622 non-null  object 
 14  orb_v             14622 non-null  object 
 15  orb_h             14622 non-null  object 
 16  ft_fga_v          14622 non-null  object

Convert date to pd.datetime object. May help with building season dictionary later.

In [201]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

In [202]:
df['Date']

0       2007-10-30
1       2007-10-30
2       2007-10-30
3       2007-10-31
4       2007-10-31
           ...    
15010   2019-06-02
15011   2019-06-05
15012   2019-06-07
15013   2019-06-10
15014   2019-06-13
Name: Date, Length: 14622, dtype: datetime64[ns]

In [203]:
d = df.reindex(df.index.repeat(2)).reset_index(drop=True)

Now that we have every game twice, we can make a new column to hold home/visitor every other row, and then rename the _h and _v columns for self, opponent.

In [204]:
d

,Date,O/U_open,O/U_close,Season,id,total,away,home,pace_v,pace_h,...,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,ou1,ou2,Over/Under_open,Over/Under_close
0,2007-10-30,184.0,189.5,0708,20071030SanAntonio,203,POR,SAS,91.8,91.8,...,50.0,51.2,8.7,6.2,100.0,105.7,19.0,13.5,1,1
1,2007-10-30,184.0,189.5,0708,20071030SanAntonio,203,POR,SAS,91.8,91.8,...,50.0,51.2,8.7,6.2,100.0,105.7,19.0,13.5,1,1
2,2007-10-30,214.5,212.0,0708,20071030GoldenState,213,UTA,GSW,105.1,105.1,...,39.8,59.4,7.6,11.4,100.0,111.3,-1.5,1.0,0,1
3,2007-10-30,214.5,212.0,0708,20071030GoldenState,213,UTA,GSW,105.1,105.1,...,39.8,59.4,7.6,11.4,100.0,111.3,-1.5,1.0,0,1
4,2007-10-30,191.0,199.0,0708,20071030LALakers,188,HOU,LAL,93.0,93.0,...,43.0,56.3,17.2,5.8,100.0,102.1,-3.0,-11.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29239,2019-06-07,216.0,215.0,1819,20190607GoldenState,197,TOR,GSW,94.8,94.8,...,51.9,74.3,6.3,11.1,100.0,110.7,-19.0,-18.0,0,0
29240,2019-06-10,212.0,217.0,1819,20190610Toronto,211,GSW,TOR,94.3,94.3,...,53.8,50.0,6.4,12.5,100.0,112.4,-1.0,-6.0,0,0
29241,2019-06-10,212.0,217.0,1819,20190610Toronto,211,GSW,TOR,94.3,94.3,...,53.8,50.0,6.4,12.5,100.0,112.4,-1.0,-6.0,0,0
29242,2019-06-13,211.0,211.5,1819,20190613GoldenState,224,TOR,GSW,94.6,94.6,...,51.9,71.8,9.5,12.2,100.0,120.5,13.0,12.5,1,1


Let's create the column we want to place our values, then selectively copy.

In [205]:
d['team'] = np.nan

In [206]:
#home team will be all even indexes of this dataset
d['team'][::2] = d['home'][::2]

<ipython-input-206-a0ddae015092>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['team'][::2] = d['home'][::2]


In [207]:
#away team will be all odd indexes of this dataset
d['team'][1::2] = d['away'][1::2]

<ipython-input-207-a4c97a9c0b76>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['team'][1::2] = d['away'][1::2]


In [208]:
d['opp'] = np.nan

In [209]:
d['opp'][::2] = d['away'][::2]
d['opp'][1::2] = d['home'][1::2]

<ipython-input-209-f37a04d04529>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['opp'][::2] = d['away'][::2]
<ipython-input-209-f37a04d04529>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['opp'][1::2] = d['home'][1::2]


In [210]:
d[['team','opp']]

,team,opp
0,SAS,POR
1,POR,SAS
2,GSW,UTA
3,UTA,GSW
4,LAL,HOU
...,...,...
29239,TOR,GSW
29240,TOR,GSW
29241,GSW,TOR
29242,GSW,TOR


Let's keep track of home/away, can't hurt. We'll use the same process as we just used.

In [211]:
d['home/away'] = np.nan
#home team will be all even indexes of this dataset
d['home/away'][::2] = 'H'
#away team will be all odd indexes of this dataset
d['home/away'][1::2] = 'V'

<ipython-input-211-915be6e0f5cd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['home/away'][::2] = 'H'
<ipython-input-211-915be6e0f5cd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['home/away'][1::2] = 'V'


In [213]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29244 entries, 0 to 29243
Data columns (total 79 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              29244 non-null  datetime64[ns]
 1   O/U_open          29244 non-null  float64       
 2   O/U_close         29244 non-null  float64       
 3   Season            29244 non-null  object        
 4   id                29244 non-null  object        
 5   total             29244 non-null  int64         
 6   away              29244 non-null  object        
 7   home              29244 non-null  object        
 8   pace_v            29244 non-null  object        
 9   pace_h            29244 non-null  object        
 10  eFg_v             29244 non-null  object        
 11  eFg_h             29244 non-null  object        
 12  tov_v             29244 non-null  object        
 13  tov_h             29244 non-null  object        
 14  orb_v             2924

Now we have to map the _v and _h stats appropriate to new columns for self or opponent stats.

In [214]:
stats = ['pace','eFg','tov','orb','ft_fga','ortg','fg','fga','fg_per','threes','threes_att','threes_per',
         'ft','fta','ft_per','drb','trb','ast','stl','blk','to','fouls','ts_per','threes_ar','ft_ar',
         'drb_per','trb_per','ast_per','stl_per','blk_per','user_per','drtg']
for stat in stats:
    d['{}'.format(stat)] = ""
    d['{}'.format(stat)][::2] = d['{}_h'.format(stat)][::2]
    d['{}'.format(stat)][1::2] = d['{}_v'.format(stat)][1::2]
    d['{}_opp'.format(stat)] = ""
    d['{}_opp'.format(stat)][::2] = d['{}_v'.format(stat)][::2]
    d['{}_opp'.format(stat)][1::2] = d['{}_h'.format(stat)][1::2]

<ipython-input-214-7aa6dc08b0bf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['{}'.format(stat)][::2] = d['{}_h'.format(stat)][::2]
<ipython-input-214-7aa6dc08b0bf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['{}'.format(stat)][1::2] = d['{}_v'.format(stat)][1::2]
<ipython-input-214-7aa6dc08b0bf>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['{}_opp'.format(stat)][::2] = d['{}_v'.format(stat)][::2]
<ipython-input-214-

In [215]:
mylist = []
for stat in stats:
    mylist.append('{}_v'.format(stat))
    mylist.append('{}_h'.format(stat))
#print(mylist)
d = d.drop(columns=mylist)

In [216]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29244 entries, 0 to 29243
Data columns (total 79 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              29244 non-null  datetime64[ns]
 1   O/U_open          29244 non-null  float64       
 2   O/U_close         29244 non-null  float64       
 3   Season            29244 non-null  object        
 4   id                29244 non-null  object        
 5   total             29244 non-null  int64         
 6   away              29244 non-null  object        
 7   home              29244 non-null  object        
 8   ou1               29244 non-null  float64       
 9   ou2               29244 non-null  float64       
 10  Over/Under_open   29244 non-null  int64         
 11  Over/Under_close  29244 non-null  int64         
 12  team              29244 non-null  object        
 13  opp               29244 non-null  object        
 14  home/away         2924

Now our DataFrame (d) has stats for each team and their opponent for every game. Now let's get some rolling averages.

## Scratch Work Graveyard

In [ ]:
#tried self-merging on game_id, but this is only applicable if in different columns
#df1 = df1.merge(
#            right=df1[opp_pull_cols],
#            left_on=["game_id", "team"],
#            right_on=["game_id", "opp"],
#            suffixes=[None, "_opp"],
#        )

In [ ]:
#tried to use drop_duplicates method but can only keep first or last
#df.drop_duplicates(subset=['game_id'], keep='second')